In [1]:
# Initialize python libs & SQL creads
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import pandas as pd
from pandasql import sqldf
import streamlit as st
import pull_nrg_data
import json
import http.client
import certifi
import ssl
import os
import timeit
import pickle
import altair as alt
import pytz
import numpy as np
tz = pytz.timezone('America/Edmonton')

In [11]:
def release_token(accessToken):
    path = '/api/ReleaseToken'
    server = 'api.nrgstream.com'
    headers = {'Authorization': f'Bearer {accessToken}'}
    context = ssl.create_default_context(cafile=certifi.where())
    conn = http.client.HTTPSConnection(server,context=context)
    conn.request('DELETE', path, None, headers)
    res = conn.getresponse()

with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)

release_token(default_pickle['accessToken'])

In [12]:
t = datetime.now(pytz.timezone('America/Edmonton')).date() - timedelta(days=8)
t2 = datetime.now(pytz.timezone('America/Edmonton')) - timedelta(days=8)
default_pickle = {
    'accessToken':'',
    'current_data':'',
    'daily_outage_dfs':[0]*7,
    'monthly_outage_dfs':[0]*7,
    'alert_dates':{
        'Biomass & Other':t, 
        'Coal':t,
        'Dual Fuel':t,
        'Energy Storage':t,
        'Hydro':t,
        'Natural Gas':t,
        'Solar':t,
        'Wind':t,
        'Intertie':t
    }
}
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)
# Add default current values to dict
with open('./current_df.pickle', 'rb') as handle:
    current_df = pickle.load(handle)
default_pickle['current_data'] = current_df
# Add default monthly values to dict
with open('./monthly_df.pickle', 'rb') as handle:
    monthly = pickle.load(handle)
for i in range(7):
    default_pickle['monthly_outage_dfs'][i] = (t2,monthly)
# Add default daily values to dict
with open('./daily_df.pickle', 'rb') as handle:
    daily = pickle.load(handle)
for i in range(7):
    default_pickle['daily_outage_dfs'][i] = (t2,daily)
with open('./default_pickle.pickle', 'wb') as handle:
    pickle.dump(default_pickle, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [5]:
with open('./default_pickle.pickle', 'rb') as handle:
    default_pickle = pickle.load(handle)
default_pickle

{'accessToken': '3uin4fPJWRnzMKz1IQUIn2Gs9sYI_hgZrZgT_Z9-RQWNBAr8E9iUO120UldIBmfoE11e3X5BwjYkaeZgm5EpTlo9JC9XYjSVSwOtvN9IxYMWbqttPIy1aPddHHIJp3cRlbyVqbneyUw4zNrojvkAyMd6iJDxBwBtUlxWM1Lu5sYb1KG6GPxWUDUCKvfwOOgvwJ0451xT2UNdccT6LNqF8Z2UCAcJWIuBqz4CYicLB_Itd60auVRcVpFIfFJX3Rj_V9TACJMo7dAsYdIS0nSPKVpzV8M4_MxVmhxwl2r1vkJCTd_nhF1Po5Jmp6LnII8Ri13ni3ys9Eb8dlOn8MzAPwmC4mU',
 'current_data': (datetime.datetime(2022, 9, 22, 14, 15, 1, 516674, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                      timeStamp    value    fuelType
  0   2022-09-22 00:00:00-06:00  1265.00        Coal
  1   2022-09-22 00:05:00-06:00  1267.00        Coal
  2   2022-09-22 00:10:00-06:00  1264.00        Coal
  3   2022-09-22 00:15:00-06:00  1262.00        Coal
  4   2022-09-22 00:20:00-06:00  1261.00        Coal
  ..                        ...      ...         ...
  166 2022-09-22 13:50:00-06:00   268.77  Pool Price
  167 2022-09-22 13:55:00-06:00   268.77  Pool Price
  168 2022-09-22 14:00:0

In [26]:
default_pickle['monthly_outage_dfs'][1]

(datetime.datetime(2022, 9, 12, 14, 0, 34, 184434, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                    timeStamp  value         fuelType
 9  2022-10-01 00:00:00-06:00    470             Coal
 10 2022-11-01 00:00:00-06:00     30             Coal
 11 2022-12-01 00:00:00-07:00      0             Coal
 9  2022-10-01 00:00:00-06:00   1140      Natural Gas
 10 2022-11-01 00:00:00-06:00    740      Natural Gas
 ..                       ...    ...              ...
 4  2024-05-01 00:00:00-06:00     30  Biomass & Other
 5  2024-06-01 00:00:00-06:00     30  Biomass & Other
 6  2024-07-01 00:00:00-06:00     30  Biomass & Other
 7  2024-08-01 00:00:00-06:00     30  Biomass & Other
 8  2024-09-01 00:00:00-06:00     30  Biomass & Other
 
 [192 rows x 3 columns])

In [28]:
with open('./monthly_df.pickle', 'wb') as daily:
    pickle.dump(default_pickle['monthly_outage_dfs'], daily, protocol=pickle.HIGHEST_PROTOCOL)

In [29]:
with open('./monthly_df.pickle', 'rb') as monthly:
    monthly_df = pickle.load(monthly)
monthly_df

[(datetime.datetime(2022, 9, 4, 13, 52, 29, 897614, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                     timeStamp  value         fuelType
  9  2022-10-01 00:00:00-06:00    470             Coal
  10 2022-11-01 00:00:00-06:00     30             Coal
  11 2022-12-01 00:00:00-07:00      0             Coal
  9  2022-10-01 00:00:00-06:00   1140      Natural Gas
  10 2022-11-01 00:00:00-06:00    740      Natural Gas
  ..                       ...    ...              ...
  4  2024-05-01 00:00:00-06:00     30  Biomass & Other
  5  2024-06-01 00:00:00-06:00     30  Biomass & Other
  6  2024-07-01 00:00:00-06:00     30  Biomass & Other
  7  2024-08-01 00:00:00-06:00     30  Biomass & Other
  8  2024-09-01 00:00:00-06:00     30  Biomass & Other
  
  [192 rows x 3 columns]),
 (datetime.datetime(2022, 9, 12, 14, 0, 34, 184434, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                     timeStamp  value         fuelType
  9  2022-10-01 00:00:0

In [33]:
monthly_df[6]

(datetime.datetime(2022, 9, 22, 14, 12, 37, 918415, tzinfo=<DstTzInfo 'America/Edmonton' MDT-1 day, 18:00:00 DST>),
                    timeStamp  value         fuelType
 9  2022-10-01 00:00:00-06:00    470             Coal
 10 2022-11-01 00:00:00-06:00     50             Coal
 11 2022-12-01 00:00:00-07:00      0             Coal
 9  2022-10-01 00:00:00-06:00   1140      Natural Gas
 10 2022-11-01 00:00:00-06:00    740      Natural Gas
 ..                       ...    ...              ...
 4  2024-05-01 00:00:00-06:00     30  Biomass & Other
 5  2024-06-01 00:00:00-06:00     30  Biomass & Other
 6  2024-07-01 00:00:00-06:00     30  Biomass & Other
 7  2024-08-01 00:00:00-06:00     30  Biomass & Other
 8  2024-09-01 00:00:00-06:00     30  Biomass & Other
 
 [192 rows x 3 columns])